## Ezyme-Subtrate Kinetics Rule Creator


This notebook uses the Boolean_rules_creator tool available to generate
rules for a boolean network describing Michaelis-Menten enzyme-substrate kinetics. For more information on the tool and installation instructions, view the GitHub page at https://github.com/LoLab-VU/Boolean_rules_creator .

   >More details on the algorithm and a description of the model are available in the preprint at
   >https://www.biorxiv.org/content/10.1101/2020.12.15.422874v1.full (Prugger et. al. 2020).


### Importing libraries


In [1]:
from rule_creator import creating_rules
import json
import sympy
from sympy.logic.boolalg import Xor
from sympy import symbols

### Loading the input file

For the ES model, there is only one possibly steady state outcome, so the Boolean ruleset can be generation without optimization. The list  In this example, only the rule_creator function is used to recreate the forward, backwards, and expert-guided rulesets.

The input file for this model is 'ES_steady_states.json', which contains the possible initial boolean states for the four species: Enzyme (En), Substrate (Su), Enzyme-Substrate complex (ES), and Product (Pr). For this script, these states are always referenced in this order.

In [2]:
fn = 'ES_steady_states.json'
En, Su, ES, Pr = symbols('En Su ES Pr')
symbols_list = ['En', 'Su', 'ES', 'Pr']

The input file contains pairs of intial and steady states followed by the frequency
with which the intial state leads to that final state. 

In [3]:
with open(fn) as fs:
    data = json.load(fs)
    fs.close()
print(data.items())

dict_items([('[0, 0, 0, 0]', [[[0, 0, 0, 0], 100]]), ('[0, 0, 0, 1]', [[[0, 0, 0, 1], 100]]), ('[0, 0, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[0, 0, 1, 1]', [[[1, 0, 0, 1], 100]]), ('[0, 1, 0, 0]', [[[0, 1, 0, 0], 100]]), ('[0, 1, 0, 1]', [[[0, 1, 0, 1], 100]]), ('[0, 1, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[0, 1, 1, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 0, 0, 0]', [[[1, 0, 0, 0], 100]]), ('[1, 0, 0, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 0, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[1, 0, 1, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 0, 0]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 0, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 1, 1]', [[[1, 0, 0, 1], 100]])])


### Generating network rules

The function creating_rules will take these input states and generate a list of rules which contain
the possible transitions to describe the boolean network. For this model, the first three states, as seen below, can only progress to their respective steady states due to the lack of reactive species. For all other intial states the only possible steady state is [1,0,0,1].

In [16]:
#Call the creating_rules function to generate str_rules and simple_rulelist
#Inputs are the filename, list of symbols, and backwardspaths (0 = forward paths, 1 = backward paths)
str_rules, simple_rulelist, fs_cpp_name = creating_rules(fn,symbols_list,backwardpaths=0)

generating rules took  0.008001089096069336 seconds


Looking at the resulting rule list with all backwords transitions included:

In [17]:
print('The string rules describe the full ruleset in formal logical terms, showing what states can activate that species.')
print('\nString rules:')
print(str_rules)

print('The simple rulelist show the string rules in their boolean form; this conveys the same information as above.')
print('\nSimple rulelist')
print(simple_rulelist)

The string rules describe the full ruleset in formal logical terms, showing what states can activate that species.

String rules:
1: En* = Xor((( not En and  not Su and ES and  not Pr) or ( not En and  not Su and ES and Pr) or ( not En and Su and ES and  not Pr) or ( not En and Su and ES and Pr)), En)
1: Su* = Xor((( not En and Su and ES and Pr) or (En and Su and ES and  not Pr) or ( not En and Su and ES and  not Pr) or (En and Su and  not ES and Pr) or (En and Su and ES and Pr)), Su)
1: ES* = Xor(((En and Su and ES and  not Pr) or (En and Su and ES and Pr) or (En and  not Su and ES and Pr)), ES)
1: Pr* = Xor((( not En and  not Su and ES and  not Pr) or (En and Su and  not ES and  not Pr) or (En and Su and ES and  not Pr) or ( not En and Su and ES and  not Pr) or (En and  not Su and ES and  not Pr)), Pr)

The simple rulelist show the string rules in their boolean form; this conveys the same information as above.

Simple rulelist
[[(0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 1, 0), (0, 1, 1, 1)]

The resulting rulelist generated using forward rules for each species is displayed in the table below based on their Hamming distance, where d is the number of steps needed to reach the steady state [1,0,0,1].

| Species 	| list for d=1  	| list for d=2         	        | list for d=3         	    | list for d=4 	|
|:---------:|:--------------: 	|:-----------------------------:|:-------------------------:|:-------------:|
| En      	|     	    |  [0,0,1,1]  | [0,1,1,1], [0,0,1,0] 	    | [0,1,1,0]    	|
| Su      	|  [1,1,0,1]     |  [1,1,1,1],                     | [1,1,1,0], [0,1,1,1],  	    | [0,1,1,0]    	|
| ES      	| [1,0,1,1]    	    |  [1,1,1,1]            	    | [1,1,1,0]            	    |              	|
| Pr      	|          	    |  [1,0,1,0], [1,1,0,0],  	    | [0,0,1,0], [1,1,1,0]  	    | [0,1,1,0]   	|



This can also be done with all backward paths included:

| Species 	| list for d=1  	| list for d=2         	        | list for d=3         	    | list for d=4 	|
|:---------:|:--------------: 	|:-----------------------------:|:-------------------------:|:-------------:|
| En      	| [1,0,1,1]    	    |  [0,0,1,1], [1,0,1,0], [1,1,1,1]   | [0,1,1,1], [1,1,1,0], [0,0,1,0] 	    | [0,1,1,0]    	|
| Su      	| [1,0,1,1], [1,1,0,1]     |  [1,1,1,1], [1,0,1,0], [0,0,1,1]                    | [1,1,1,0], [0,1,1,1], [0,0,1,0] 	    | [0,1,1,0]    	|
| ES      	| [1,0,1,1], [1,1,0,1]    	    |  [1,1,0,0], [1,1,1,1]            	    | [1,1,1,0]            	    |              	|
| Pr      	|  [1,0,1,1], [1,1,0,1]           	    |  [1,0,1,0], [1,1,0,0], [0,0,1,1], [1,1,1,1]	    | [0,0,1,0], [1,1,1,0], [0,1,1,1]  	    | [0,1,1,0]    	|




With the input variable backwardspath set as 1, we generate the network with all possible backwards paths available
for transitions. The simple rulelist output describes the resulting network. The example below (from Figure 3) shows the network formed with the initial state [1,1,0,0].


![alt text](bw_1100_4.png "ES-B Network")

### Generating simplified rules from manuscript

In order to simplify the forward only rules to match those in the paper, the Sympy library is used. For this, the string rules must first be converted to formal logic notation.

In [6]:
str_rules_list = str_rules.split('\n')
str_rules_list = str_rules_list[0:4]
count = 0

for n in str_rules_list: 
    str_rules_list[count] = n.strip('1: '+symbols_list[count]+'* = ') 
    count +=1

sympy_input = []
for k in range(len(str_rules_list)):
    sympy_input.append(str_rules_list[k].replace('and', '&').replace(' or',' |').replace('not ', '~'))

print(sympy_input)

['Xor((( ~En &  ~Su & ES &  ~Pr) | ( ~En &  ~Su & ES & Pr) | ( ~En & Su & ES &  ~Pr) | ( ~En & Su & ES & Pr)), En)', 'Xor((( ~En & Su & ES & Pr) | (En & Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr) | (En & Su &  ~ES & Pr) | (En & Su & ES & Pr)), Su)', 'Xor(((En & Su & ES &  ~Pr) | (En & Su & ES & Pr) | (En &  ~Su & ES & Pr)), ES)', 'Xor((( ~En &  ~Su & ES &  ~Pr) | (En & Su &  ~ES &  ~Pr) | (En & Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr) | (En &  ~Su & ES &  ~Pr)), Pr)']


Using Sympy to simplify the Xor statement for each species yields the rules for the ES-B network as seen in the paper. Note that '|' represents 'or', '&' represents 'and', and '~' represents 'not'.

In [7]:
print('Rules after simplifcation with Sympy')
#exp_EN = sympy.simplify(sympy.sympify(sympy_input[0][1]))

print('EN* = ' + str(sympy.to_dnf(sympy_input[0], simplify=True)))
print('S* = ' + str(sympy.to_dnf(sympy_input[1], simplify=True)))
print('ES* = ' + str(sympy.to_dnf(sympy_input[2], simplify=True)))
print('P* = ' + str(sympy.to_dnf(sympy_input[3], simplify=True)))


Rules after simplifcation with Sympy
EN* = ES | En
S* = (Su & ~ES & ~En) | (Su & ~ES & ~Pr)
ES* = (ES & ~En) | (ES & ~Pr & ~Su)
P* = ES | Pr | (En & Su)


### Incorporating expert knowledge

Now we are demonstrating how expert knowledge can be incorporated so that we get the correct kinetics. Here we are adding two transitions we know to be necessary and recreating the rule list using these with additions with the forward paths only.

In [8]:
addlist = [((1, 1, 1, 0), 0), ((1, 1, 0, 0), 2)]

str_rules_expert_fw, simple_rulelist_expert_fw, fs_cpp_name_expert_fw = creating_rules(fn,symbols_list,backwardpaths=0,addlist=addlist)

generating rules took  0.010995149612426758 seconds


Formatting the string rules for simplification by Sympy.

In [9]:
str_rules_list = str_rules_expert_fw.split('\n')
str_rules_list = str_rules_list[0:4]
count = 0

for n in str_rules_list: 
    str_rules_list[count] = n.strip('1: '+symbols_list[count]+'* = ') 
    count +=1

sympy_input = []
for k in range(len(str_rules_list)):
    sympy_input.append(str_rules_list[k].replace('and', '&').replace(' or',' |').replace('not ', '~'))

print(sympy_input)

['Xor((( ~En &  ~Su & ES &  ~Pr) | ( ~En & Su & ES & Pr) | (En & Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr) | ( ~En &  ~Su & ES & Pr)), En)', 'Xor((( ~En & Su & ES & Pr) | (En & Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr) | (En & Su &  ~ES & Pr) | (En & Su & ES & Pr)), Su)', 'Xor(((En & Su &  ~ES &  ~Pr) | (En & Su & ES &  ~Pr) | (En & Su & ES & Pr) | (En &  ~Su & ES & Pr)), ES)', 'Xor((( ~En &  ~Su & ES &  ~Pr) | (En & Su &  ~ES &  ~Pr) | (En & Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr) | (En &  ~Su & ES &  ~Pr)), Pr)']


In [10]:
print('Rules after simplifcation with Sympy')
#exp_EN = sympy.simplify(sympy.sympify(sympy_input[0][1]))

print('EN* = ' + str(sympy.to_dnf(sympy_input[0], simplify=True)))
print('S* = ' + str(sympy.to_dnf(sympy_input[1], simplify=True)))
print('ES* = ' + str(sympy.to_dnf(sympy_input[2], simplify=True)))
print('P* = ' + str(sympy.to_dnf(sympy_input[3], simplify=True)))


Rules after simplifcation with Sympy
EN* = (ES & Pr) | (ES & ~En) | (ES & ~Su) | (En & ~ES)
S* = (Su & ~ES & ~En) | (Su & ~ES & ~Pr)
ES* = (ES & ~En) | (ES & ~Pr & ~Su) | (En & Su & ~ES & ~Pr)
P* = ES | Pr | (En & Su)


We can now see the transitions based on Hamming distance for these expert guided forward path rules.

| species 	| list for d=1  	| list for d=2         	        | list for d=3         	    | list for d=4 	|
|:---------:|:--------------: 	|:-----------------------------:|:-------------------------:|:-------------:|
| En      	|              	    |  [0,0,1,1]            	    | [0,0,1,0], [0,1,1,1] 	    | [0,1,1,0]    	|
| Su      	| [1,1,0,1]    	    |  [1,1,1,1]                    | [1,1,1,0], [0,1,1,1] 	    | [0,1,1,0]    	|
| ES      	| [1,0,1,1]    	    |  [1,1,1,1], [1,1,0,0]            	    | [1,1,1,0]            	    |              	|
| Pr      	|              	    |  [1,0,1,0], [1,1,0,0] 	    | [1,1,1,0], [0,0,1,0] 	    | [0,1,1,0]    	|


### Expert Backwards Rules

Finally, to demonstrate the creation of the ES-E expert guided network from the paper, we remove transitions known to be false from the rulelist by including them in the blacklist variable.

In [11]:
blacklist = [((1, 0, 1, 1), 0), ((1, 0, 1, 0), 0), ((1, 1, 1, 1), 0),
             ((1, 0, 1, 1), 1), ((1, 0, 1, 0), 1), ((0, 0, 1, 0), 1), ((0, 0, 1, 1), 1),
             ((1, 1, 0, 1), 2),
             ((1, 0, 1, 1), 3), ((1, 1, 0, 1), 3), ((0, 1, 1, 1), 3), ((1, 1, 1, 1), 3), ((0, 0, 1, 1), 3)]

str_rules_expert_bw, simple_rulelist_expert_bw, fs_cpp_name_expert_bw = creating_rules(fn,symbols_list,backwardpaths=1,blacklist=blacklist)

generating rules took  0.012009859085083008 seconds


Formatting the string rules for simplification by Sympy.

In [12]:
str_rules_list = str_rules_expert_bw.split('\n')
str_rules_list = str_rules_list[0:4]
count = 0

for n in str_rules_list: 
    str_rules_list[count] = n.strip('1: '+symbols_list[count]+'* = ') 
    count +=1

sympy_input = []
for k in range(len(str_rules_list)):
    sympy_input.append(str_rules_list[k].replace('and', '&').replace(' or',' |').replace('not ', '~'))

print(sympy_input)

['Xor((( ~En &  ~Su & ES &  ~Pr) | ( ~En & Su & ES & Pr) | (En & Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr) | ( ~En &  ~Su & ES & Pr)), En)', 'Xor((( ~En & Su & ES & Pr) | (En & Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr) | (En & Su &  ~ES & Pr) | (En & Su & ES & Pr)), Su)', 'Xor(((En & Su &  ~ES &  ~Pr) | (En & Su & ES &  ~Pr) | (En & Su & ES & Pr) | (En &  ~Su & ES & Pr)), ES)', 'Xor((( ~En &  ~Su & ES &  ~Pr) | (En & Su &  ~ES &  ~Pr) | (En & Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr) | (En &  ~Su & ES &  ~Pr)), Pr)']


In [13]:
print('Rules after simplifcation with Sympy')
#exp_EN = sympy.simplify(sympy.sympify(sympy_input[0][1]))

print('EN* = ' + str(sympy.to_dnf(sympy_input[0], simplify=True)))
print('S* = ' + str(sympy.to_dnf(sympy_input[1], simplify=True)))
print('ES* = ' + str(sympy.to_dnf(sympy_input[2], simplify=True)))
print('P* = ' + str(sympy.to_dnf(sympy_input[3], simplify=True)))


Rules after simplifcation with Sympy
EN* = (ES & Pr) | (ES & ~En) | (ES & ~Su) | (En & ~ES)
S* = (Su & ~ES & ~En) | (Su & ~ES & ~Pr)
ES* = (ES & ~En) | (ES & ~Pr & ~Su) | (En & Su & ~ES & ~Pr)
P* = ES | Pr | (En & Su)


You may notice that this rule list is the same as that generated from the Expert Forwards rulelist.

| species 	| list for d=1  	| list for d=2         	        | list for d=3         	    | list for d=4 	|
|:---------:|:--------------: 	|:-----------------------------:|:-------------------------:|:-------------:|
| En      	|              	    |  [0,0,1,1]            	    | [0,1,1,1], [1,1,1,0], [0,0,1,0] 	    | [0,1,1,0]    	|
| Su      	| [1,1,0,1]    	    |  [1,1,1,1]                    | [1,1,1,0], [0,1,1,1] 	    | [0,1,1,0]    	|
| ES      	| [1,0,1,1]    	    |  [1,1,1,1], [1,1,0,0]            	    | [1,1,1,0]            	    |              	|
| Pr      	|              	    |  [1,0,1,0], [1,1,0,0] 	    | [1,1,1,0], [0,0,1,0] 	    | [0,1,1,0]    	|

Although the correct kinetics can be modeled by both adding transitions to the foward paths or removing transitions from the backward paths, it is recommended to use the backward paths approach as the forward paths approach can result in losing attractor states or creating new false attractors. With the backward paths approach, we only need to ensure that transitions necessary for states to reach the attractor are not removed.